In [26]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import missingno as msno

from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, normalize
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


from sklearn.model_selection import cross_validate

from sklearn.model_selection import ParameterGrid

import warnings
warnings.filterwarnings('ignore')

import math

from sklearn import ensemble

pd.set_option('display.max_columns', None)

%matplotlib inline  

housing_raw = pd.read_csv('./data/Ames_Housing_Price_Data.csv', index_col=0)
price = housing_raw['SalePrice'].copy()
housing_raw

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal
2,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal
3,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal
4,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal
5,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2001.0,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,903205040,952,121000,30,RL,NaN,8854,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,BrkSide,Norm,Norm,1Fam,1.5Unf,6,6,1916,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,952.0,952.0,Grav,Fa,N,FuseF,952,0,0,0.0,0.0,1,0,2,1,Fa,4,Typ,1,Gd,Detchd,1916.0,Unf,1.0,192.0,Fa,Po,P,0,98,0,0,40,0,NaN,NaN,NaN,0,5,2009,WD,Normal
760,905402060,1733,139600,20,RL,NaN,13680,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Edwards,Norm,Norm,1Fam,1Story,3,5,1955,1955,Hip,CompShg,BrkFace,Wd Sdng,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,GasA,Ex,Y,FuseA,1733,0,0,0.0,0.0,2,0,4,1,TA,8,Min2,1,Gd,Attchd,1955.0,Unf,2.0,452.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal
761,909275030,2002,145000,90,RH,82.0,6270,Pave,NaN,Reg,HLS,AllPub,Inside,Gtl,Crawfor,Norm,Norm,Duplex,2Story,5,6,1949,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,284.0,Unf,0.0,717.0,1001.0,GasA,TA,N,FuseA,1001,1001,0,0.0,0.0,2,0,4,2,TA,8,Typ,0,NaN,2Types,1949.0,Unf,3.0,871.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
762,907192040,1842,217500,60,RL,N

In [27]:
housing_raw = housing_raw[housing_raw['GrLivArea']< 4000]

In [28]:
df_num = housing_raw.select_dtypes(['int64','float64']).copy()
df_num.head()

,PID,GrLivArea,SalePrice,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
1,909176150,856,126000,30,NaN,7890,6,6,1939,1950,0.0,238.0,0.0,618.0,856.0,856,0,0,1.0,0.0,1,0,2,1,4,1,1939.0,2.0,399.0,0,0,0,0,166,0,0,3,2010
2,905476230,1049,139500,120,42.0,4235,5,5,1984,1984,149.0,552.0,393.0,104.0,1049.0,1049,0,0,1.0,0.0,2,0,2,1,5,0,1984.0,1.0,266.0,0,105,0,0,0,0,0,2,2009
3,911128020,1001,124900,30,60.0,6060,5,9,1930,2007,0.0,737.0,0.0,100.0,837.0,1001,0,0,0.0,0.0,1,0,2,1,5,0,1930.0,1.0,216.0,154,0,42,86,0,0,0,11,2007
4,535377150,1039,114000,70,80.0,8146,4,8,1900,2003,0.0,0.0,0.0,405.0,405.0,717,322,0,0.0,0.0,1,0,2,1,6,0,1940.0,1.0,281.0,0,0,168,0,111,0,0,5,2009
5,534177230,1665,227000,60,70.0,8400,8,6,2001,2001,0.0,643.0,0.0,167.0,810.0,810,855,0,1.0,0.0,2,1,3,1,6,0,2001.0,2.0,528.0,0,45,0,0,0,0,0,11,2009


In [29]:
df_num['LotFrontage'].fillna(df_num['LotFrontage'].mean(), inplace = True)

# Because there is only 1 null value for area, we just fill in with the Year the house was built
df_num.loc[df_num['GarageArea'].isnull(), 'GarageYrBlt'] = df_num.loc[df_num['GarageArea'].isnull(),'YearBuilt']
df_num['GarageArea'].fillna(housing_raw[housing_raw['GarageType']== 'Detchd']['GarageArea'].mean(), inplace = True)
df_num.loc[df_num['GarageCars'].isnull(), 'GarageCars'] = 2


df_num.drop(['SalePrice', 'PID'],inplace=True, axis = 1)
# MsnrVrArea, TotalBsmtSF, GarageCars, BsmtHalfBath, BsmtFullBath
df_num.fillna(0,inplace = True)
price = housing_raw['SalePrice'].copy()
# msno.bar(df_num)
df_num

,GrLivArea,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
1,856,30,68.470227,7890,6,6,1939,1950,0.0,238.0,0.0,618.0,856.0,856,0,0,1.0,0.0,1,0,2,1,4,1,1939.0,2.0,399.0,0,0,0,0,166,0,0,3,2010
2,1049,120,42.000000,4235,5,5,1984,1984,149.0,552.0,393.0,104.0,1049.0,1049,0,0,1.0,0.0,2,0,2,1,5,0,1984.0,1.0,266.0,0,105,0,0,0,0,0,2,2009
3,1001,30,60.000000,6060,5,9,1930,2007,0.0,737.0,0.0,100.0,837.0,1001,0,0,0.0,0.0,1,0,2,1,5,0,1930.0,1.0,216.0,154,0,42,86,0,0,0,11,2007
4,1039,70,80.000000,8146,4,8,1900,2003,0.0,0.0,0.0,405.0,405.0,717,322,0,0.0,0.0,1,0,2,1,6,0,1940.0,1.0,281.0,0,0,168,0,111,0,0,5,2009
5,1665,60,70.000000,8400,8,6,2001,2001,0.0,643.0,0.0,167.0,810.0,810,855,0,1.0,0.0,2,1,3,1,6,0,2001.0,2.0,528.0,0,45,0,0,0,0,0,11,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,952,30,68.470227,8854,6,6,1916,1950,0.0,0.0,0.0,952.0,952.0,952,0,0,0.0,0.0,1,0,2,1,4,1,1916.0,1.0,192.0,0,98,0,0,40,0,0,5,2009
760,1733,20,68.470227,13680,3,5,1955,1955,0.0,0.0,0.0,0.0,0.0,1733,0,0,0.0,0.0,2,0,4,1,8,1,1955.0,2.0,452.0,0,0,0,0,0,0,0,6,2009
761,2002,90,82.000000,6270,5,6,1949,1950,0.0,284.0,0.0,717.0,1001.0,1001,1001,0,0.0,0.0,2,0,4,2,8,0,1949.0,3.0,871.0,0,0,0,0,0,0,0,8,2007
762,1842,60,68.470227,8826,7,5,2000,2000,144.0,841.0,0.0,144.0,985.0,985,857,0,1.0,0.0,2,1,3,1,7,1,2000.0,2.0,486.0,193,96,0,0,0,0,0,7,2007


In [30]:
# Selecting Categorical Values
# Mapping the respective categorical columns ordinally
df_cat = housing_raw.select_dtypes(['object']).copy()
df_cat.fillna('None', inplace = True)
df_cat
# Define the mapping
po_ex_ordinal_mapping = {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd':4, 'Ex':5}
# Apply the mapping
df_cat['ExterQual'] = df_cat['ExterQual'].map(po_ex_ordinal_mapping)
df_cat['ExterCond'] = df_cat['ExterCond'].map(po_ex_ordinal_mapping)
df_cat['KitchenQual'] = df_cat['KitchenQual'].map(po_ex_ordinal_mapping)
df_cat['HeatingQC'] = df_cat['HeatingQC'].map(po_ex_ordinal_mapping)


# Defind the mapping
none_ex6_ordinal_mapping = {'None': 0, 'Po': 1, 'Fa': 2, 'TA':3, 'Gd':4, 'Ex':5}
# Apply the mapping
df_cat['BsmtQual'] = df_cat['BsmtQual'].map(none_ex6_ordinal_mapping)
df_cat['BsmtCond'] = df_cat['BsmtCond'].map(none_ex6_ordinal_mapping)
df_cat['FireplaceQu'] = df_cat['FireplaceQu'].map(none_ex6_ordinal_mapping)
df_cat['GarageQual'] = df_cat['GarageQual'].map(none_ex6_ordinal_mapping)
df_cat['GarageCond'] = df_cat['GarageCond'].map(none_ex6_ordinal_mapping)


# Define the mapping
none_fin_ordinal_mapping = {'None': 0, 'Unf': 1, 'RFn': 2, 'Fin':3}
# Applying the mapping
df_cat['GarageFinish'] = df_cat['GarageFinish'].map(none_fin_ordinal_mapping)


# Define the mapping
none_ex5_ordinal_mapping = {'None': 0, 'Fa': 1, 'TA': 2, 'Gd':3, 'Ex':4}
# Applying the mapping
df_cat['PoolQC'] = df_cat['PoolQC'].map(none_ex5_ordinal_mapping)

# Define mapping
binary_mapping = {'Y': 1, 'N':0}
df_cat['CentralAir'] = df_cat['CentralAir'].map(binary_mapping)

# 'BsmtCond',
#  'BsmtQual',
#  'CentralAir',
#  'ExterCond',
#  'ExterQual',
#  'FireplaceQu',
#  'GarageCond',
#  'GarageFinish',
#  'GarageQual',
#  'HeatingQC',
#  'KitchenQual',
#  'PoolQC'

df_cat.shape

(2578, 43)

In [31]:
kept_columns = ['GrLivArea', 
'MSSubClass', 
'LotFrontage',
'LotArea',
'OverallQual', 
'OverallCond', 
'YearBuilt',
'YearRemodAdd', 
'MasVnrArea', 
'BsmtFinSF1', 
'BsmtFinSF2', 
'TotalBsmtSF', 
'2ndFlrSF', 
'LowQualFinSF',
'BsmtFullBath', 
'BsmtHalfBath',
'HalfBath',
'BedroomAbvGr', 
'KitchenAbvGr', 
'TotRmsAbvGrd', 
'Fireplaces', 
'GarageYrBlt',
'GarageCars', 
'GarageArea',
'WoodDeckSF', 
'3SsnPorch', 
'ScreenPorch',
'PoolArea',
'MoSold', 
'YrSold', 
'ExterQual', 
'BsmtQual', 
'BsmtCond', 
'HeatingQC', 
'CentralAir', 
'KitchenQual', 
'FireplaceQu', 
'GarageFinish',
'PoolQC',
'Neighborhood',
'BldgType',
'BsmtExposure']

kept_features2 = ['GrLivArea',
'LotFrontage',
'LotArea',
'OverallQual',
'OverallCond',
'YearBuilt',
'YearRemodAdd',
'MasVnrArea',
'BsmtFinSF1',
'BsmtFinSF2',
'TotalBsmtSF',
'2ndFlrSF',
'LowQualFinSF',
'BsmtFullBath',
'BsmtHalfBath',
'HalfBath',
'BedroomAbvGr',
'KitchenAbvGr',
'TotRmsAbvGrd',
'Fireplaces',
'GarageYrBlt',
'GarageCars',
'GarageArea',
'WoodDeckSF',
'OpenPorchSF',
'EnclosedPorch',
'3SsnPorch',
'ScreenPorch',
'PoolArea',
'MoSold',
'YrSold',
'ExterQual',
'ExterCond',
'BsmtQual',
'BsmtCond',
'HeatingQC',
'CentralAir',
'KitchenQual',
'FireplaceQu',
'GarageFinish',
'GarageQual',
'GarageCond',
'PoolQC',

'LotConfig',

'Neighborhood',



'BldgType',


'MasVnrType',

'BsmtExposure',

'BsmtFinType1',

'GarageType',
]

len(kept_columns)

42

In [32]:
df_no_dummy = pd.concat([df_num, df_cat], axis = 1)
df_no_dummy2 = df_no_dummy[kept_features2]
df_no_dummy = df_no_dummy[kept_columns]
df_no_dummy2

,GrLivArea,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,TotalBsmtSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC,LotConfig,Neighborhood,BldgType,MasVnrType,BsmtExposure,BsmtFinType1,GarageType
1,856,68.470227,7890,6,6,1939,1950,0.0,238.0,0.0,856.0,0,0,1.0,0.0,0,2,1,4,1,1939.0,2.0,399.0,0,0,0,0,166,0,3,2010,3,3,3,3,3,1,3,4,1,3,3,0,Corner,SWISU,1Fam,None,No,Rec,Detchd
2,1049,42.000000,4235,5,5,1984,1984,149.0,552.0,393.0,1049.0,0,0,1.0,0.0,0,2,1,5,0,1984.0,1.0,266.0,0,105,0,0,0,0,2,2009,4,3,4,3,3,1,4,0,3,3,3,0,Inside,Edwards,TwnhsE,BrkFace,Mn,GLQ,Attchd
3,1001,60.000000,6060,5,9,1930,2007,0.0,737.0,0.0,837.0,0,0,0.0,0.0,0,2,1,5,0,1930.0,1.0,216.0,154,0,42,86,0,0,11,2007,4,3,3,3,5,1,4,0,1,3,1,0,Inside,IDOTRR,1Fam,None,No,ALQ,Detchd
4,1039,80.000000,8146,4,8,1900,2003,0.0,0.0,0.0,405.0,322,0,0.0,0.0,0,2,1,6,0,1940.0,1.0,281.0,0,0,168,0,111,0,5,2009,4,4,2,3,4,1,3,0,1,3,3,0,Corner,OldTown,1Fam,None,No,Unf,Detchd
5,1665,70.000000,8400,8,6,2001,2001,0.0,643.0,0.0,810.0,855,0,1.0,0.0,1,3,1,6,0,2001.0,2.0,528.0,0,45,0,0,0,0,11,2009,4,3,4,3,5,1,4,0,3,3,3,0,Inside,NWAmes,1Fam,None,No,GLQ,Attchd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,952,68.470227,8854,6,6,1916,1950,0.0,0.0,0.0,952.0,0,0,0.0,0.0,0,2,1,4,1,1916.0,1.0,192.0,0,98,0,0,40,0,5,2009,3,3,3,3,2,0,2,4,1,2,1,0,Inside,BrkSide,1Fam,None,No,Unf,Detchd
760,1733,68.470227,13680,3,5,1955,1955,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,4,1,8,1,1955.0,2.0,452.0,0,0,0,0,0,0,6,2009,3,3,0,0,5,1,3,4,1,3,3,0,CulDSac,Edwards,1Fam,None,None,None,Attchd
761,2002,82.000000,6270,5,6,1949,1950,0.0,284.0,0.0,1001.0,1001,0,0.0,0.0,0,4,2,8,0,1949.0,3.0,871.0,0,0,0,0,0,0,8,2007,3,3,3,3,3,0,3,0,1,3,3,0,Inside,Crawfor,Duplex,None,No,BLQ,2Types
762,1842,68.470227,8826,7,5,2000,2000,144.0,841.0,0.0,985.0,857,0,1.0,0.0,1,3,1,7,1,2000.0,2.0,486.0,193,96,0,0,0,0,7,2007,4,3,4,3,5,1,4,3,3,3,3,0,Inside,CollgCr,1Fam,BrkFace,No,GLQ,Attchd


In [33]:
df_cat_dummy = df_no_dummy.select_dtypes('object')
df_num_dummy = df_no_dummy.select_dtypes(['int64','float64'])
df_cat_dummy = pd.get_dummies(df_cat_dummy, drop_first=True)

df_cat_dummy2 = df_no_dummy2.select_dtypes('object')
df_num_dummy2 = df_no_dummy2.select_dtypes(['int64','float64'])
df_cat_dummy2 = pd.get_dummies(df_cat_dummy2, drop_first=True)
df_cat_dummy2

,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0
2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
760,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0
761,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
762,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0


In [34]:
df = pd.concat([df_num_dummy, df_cat_dummy], axis = 1)
df2 =  pd.concat([df_num_dummy2, df_cat_dummy2], axis = 1)
df2.columns

Index(['GrLivArea', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'TotalBsmtSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath',
       'BsmtHalfBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MoSold', 'YrSold', 'ExterQual', 'ExterCond',
       'BsmtQual', 'BsmtCond', 'HeatingQC', 'CentralAir', 'KitchenQual',
       'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
       'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3',
       'LotConfig_Inside', 'Neighborhood_Blueste', 'Neighborhood_BrDale',
       'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr',
       'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert',
       'Neighborhood_Greens', 'Neighborhood_

In [35]:
df_num_unscaled = df.select_dtypes(['int64','float64'])
df_cat_dummy = df.select_dtypes('uint8')

df_num_unscaled2 = df2.select_dtypes(['int64','float64'])
df_cat_dummy2 = df2.select_dtypes('uint8')
df_num_unscaled2

,GrLivArea,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,TotalBsmtSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC
1,856,68.470227,7890,6,6,1939,1950,0.0,238.0,0.0,856.0,0,0,1.0,0.0,0,2,1,4,1,1939.0,2.0,399.0,0,0,0,0,166,0,3,2010,3,3,3,3,3,1,3,4,1,3,3,0
2,1049,42.000000,4235,5,5,1984,1984,149.0,552.0,393.0,1049.0,0,0,1.0,0.0,0,2,1,5,0,1984.0,1.0,266.0,0,105,0,0,0,0,2,2009,4,3,4,3,3,1,4,0,3,3,3,0
3,1001,60.000000,6060,5,9,1930,2007,0.0,737.0,0.0,837.0,0,0,0.0,0.0,0,2,1,5,0,1930.0,1.0,216.0,154,0,42,86,0,0,11,2007,4,3,3,3,5,1,4,0,1,3,1,0
4,1039,80.000000,8146,4,8,1900,2003,0.0,0.0,0.0,405.0,322,0,0.0,0.0,0,2,1,6,0,1940.0,1.0,281.0,0,0,168,0,111,0,5,2009,4,4,2,3,4,1,3,0,1,3,3,0
5,1665,70.000000,8400,8,6,2001,2001,0.0,643.0,0.0,810.0,855,0,1.0,0.0,1,3,1,6,0,2001.0,2.0,528.0,0,45,0,0,0,0,11,2009,4,3,4,3,5,1,4,0,3,3,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,952,68.470227,8854,6,6,1916,1950,0.0,0.0,0.0,952.0,0,0,0.0,0.0,0,2,1,4,1,1916.0,1.0,192.0,0,98,0,0,40,0,5,2009,3,3,3,3,2,0,2,4,1,2,1,0
760,1733,68.470227,13680,3,5,1955,1955,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,4,1,8,1,1955.0,2.0,452.0,0,0,0,0,0,0,6,2009,3,3,0,0,5,1,3,4,1,3,3,0
761,2002,82.000000,6270,5,6,1949,1950,0.0,284.0,0.0,1001.0,1001,0,0.0,0.0,0,4,2,8,0,1949.0,3.0,871.0,0,0,0,0,0,0,8,2007,3,3,3,3,3,0,3,0,1,3,3,0
762,1842,68.470227,8826,7,5,2000,2000,144.0,841.0,0.0,985.0,857,0,1.0,0.0,1,3,1,7,1,2000.0,2.0,486.0,193,96,0,0,0,0,7,2007,4,3,4,3,5,1,4,3,3,3,3,0


In [36]:
scaler = StandardScaler()
scaler.fit(df_num_unscaled)
scaled_num_cols = scaler.transform(df_num_unscaled)
scaled_df_num = pd.DataFrame(scaled_num_cols, columns = df_num_unscaled.columns, index = df_num_unscaled.index)
scaled_df_num

,GrLivArea,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,TotalBsmtSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,ExterQual,BsmtQual,BsmtCond,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,PoolQC
1,-1.303728,-0.646808,0.000000,-0.273213,-0.031580,0.339802,-1.053092,-1.646773,-0.564369,-0.479750,-0.305351,-0.430024,-0.794860,-0.095647,1.089629,-0.253009,-0.757088,-1.033035,-0.199131,-1.557024,0.611682,0.140002,0.343282,-0.325878,-0.742741,-0.099340,2.635518,-0.054813,-1.181068,1.646610,-0.662480,-0.510089,0.133479,-1.188897,0.259798,-0.748254,1.254384,-0.798986,-0.054241
2,-0.902872,1.455487,-1.282083,-0.724230,-0.765034,-0.551357,0.461396,0.012780,0.292857,0.254809,1.947821,0.033831,-0.794860,-0.095647,1.089629,-0.253009,-0.757088,-1.033035,-0.199131,-0.903915,-0.931289,0.244626,-1.011971,-0.967797,-0.742741,-0.099340,-0.285269,-0.054813,-1.555858,0.884635,1.129622,0.623540,0.133479,-1.188897,0.259798,0.816222,-0.987031,1.455842,-0.054241
3,-1.002567,-0.646808,-0.410255,-0.499030,-0.765034,3.013279,-1.355989,1.135419,-0.564369,0.687590,-0.305351,-0.475688,-0.794860,-0.095647,-0.838349,-0.253009,-0.757088,-1.033035,-0.199131,-0.903915,-0.931289,0.119077,-1.011971,-1.209119,0.451659,3.300038,-0.285269,-0.054813,1.817251,-0.639314,1.129622,-0.510089,0.133479,0.903237,0.259798,0.816222,-0.987031,-0.798986,-0.054241
4,-0.923642,0.287546,0.558444,-0.241624,-1.498489,2.122120,-2.365648,0.940177,-0.564369,-1.036517,-0.305351,-1.513953,-0.032572,-0.095647,-0.838349,-0.253009,-0.757088,-1.033035,-0.199131,-0.250806,-0.931289,0.142327,-1.011971,-0.895400,-0.742741,-0.099340,1.667787,-0.054813,-0.431488,0.884635,1.129622,-1.643719,0.133479,-0.142830,0.259798,-0.748254,-0.987031,-0.798986,-0.054241
5,0.376546,0.053957,0.074094,-0.210281,1.435328,0.339802,1.033536,0.842556,-0.564369,0.467690,-0.305351,-0.540580,1.229228,-0.095647,1.089629,-0.253009,1.246785,0.182467,-0.199131,-0.250806,-0.931289,0.284151,0.343282,0.296734,-0.742741,-0.099340,-0.285269,-0.054813,1.817251,0.884635,1.129622,0.623540,0.133479,0.903237,0.259798,0.816222,-0.987031,1.455842,-0.054241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,-1.104339,-0.646808,0.000000,-0.154259,-0.031580,0.339802,-1.827163,-1.646773,-0.564369,-1.036517,-0.305351,-0.199298,-0.794860,-0.095647,-0.838349,-0.253009,-0.757088,-1.033035,-0.199131,-1.557024,0.611682,0.086527,-1.011971,-1.324954,-0.742741,-0.099340,0.418535,-0.054813,-0.431488,0.884635,-0.662480,-0.510089,0.133479,-2.234964,-3.849149,-2.312730,1.254384,-0.798986,-0.054241
760,0.517780,-0.880396,0.000000,0.441256,-2.231943,-0.551357,-0.514607,-1.402721,-0.564369,-1.036517,-0.305351,-2.487327,-0.794860,-0.095647,-0.838349,-0.253009,-0.757088,1.397968,-0.199131,1.055412,0.611682,0.177202,0.343282,-0.070076,-0.742741,-0.099340,-0.285269,-0.054813,-0.056698,0.884635,-0.662480,-3.910978,-5.187781,0.903237,0.259798,-0.748254,1.254384,-0.798986,-0.054241
761,1.076487,0.754722,0.655313,-0.473117,-0.765034,0.339802,-0.716539,-1.646773,-0.564369,-0.372140,-0.305351,-0.081532,1.574862,-0.095647,-0.838349,-0.253009,-0.757088,1.397968,4.737030,1.055412,-0.931289,0.163252,1.698534,1.952207,-0.742741,-0.099340,-0.285269,-0.054813,0.692882,-0.639314,-0.662480,-0.510089,0.133479,-1.188897,-3.849149,-0.748254,-0.987031,-0.798986,-0.054241
762,0.744171,0.053957,0.000000,-0.157714,0.701874,-0.551357,0.999881,0.793746,0.264091,0.930883,-0.305351,-0.119986,1.233963,-0.095647,1.089629,-0.253009,1.246785,0.182467,-0.199131,0.402303,0.611682,0.281826,0.343282,0.094023,0.754137,-0.099340,-0.285269,-0.054813,0.318092,-0.639314,1.129622,0.623540,0.133479,0.903237,0.259798,0.816222,0.694030,1.455842,-0.054241


In [37]:
scaler = StandardScaler()
scaler.fit(df_num_unscaled2)
scaled_num_cols2 = scaler.transform(df_num_unscaled2)
scaled_df_num2 = pd.DataFrame(scaled_num_cols2, columns = df_num_unscaled2.columns, index = df_num_unscaled2.index)
scaled_df_num2

,GrLivArea,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,TotalBsmtSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC
1,-1.303728,0.000000,-0.273213,-0.031580,0.339802,-1.053092,-1.646773,-0.564369,-0.479750,-0.305351,-0.430024,-0.794860,-0.095647,1.089629,-0.253009,-0.757088,-1.033035,-0.199131,-1.557024,0.611682,0.140002,0.343282,-0.325878,-0.742741,-0.699388,-0.362343,-0.099340,2.635518,-0.054813,-1.181068,1.646610,-0.662480,-0.248625,-0.510089,0.133479,-1.188897,0.259798,-0.748254,1.254384,-0.798986,0.267572,0.257405,-0.054241
2,-0.902872,-1.282083,-0.724230,-0.765034,-0.551357,0.461396,0.012780,0.292857,0.254809,1.947821,0.033831,-0.794860,-0.095647,1.089629,-0.253009,-0.757088,-1.033035,-0.199131,-0.903915,-0.931289,0.244626,-1.011971,-0.967797,-0.742741,0.898982,-0.362343,-0.099340,-0.285269,-0.054813,-1.555858,0.884635,1.129622,-0.248625,0.623540,0.133479,-1.188897,0.259798,0.816222,-0.987031,1.455842,0.267572,0.257405,-0.054241
3,-1.002567,-0.410255,-0.499030,-0.765034,3.013279,-1.355989,1.135419,-0.564369,0.687590,-0.305351,-0.475688,-0.794860,-0.095647,-0.838349,-0.253009,-0.757088,-1.033035,-0.199131,-0.903915,-0.931289,0.119077,-1.011971,-1.209119,0.451659,-0.699388,0.292709,3.300038,-0.285269,-0.054813,1.817251,-0.639314,1.129622,-0.248625,-0.510089,0.133479,0.903237,0.259798,0.816222,-0.987031,-0.798986,0.267572,-2.653078,-0.054241
4,-0.923642,0.558444,-0.241624,-1.498489,2.122120,-2.365648,0.940177,-0.564369,-1.036517,-0.305351,-1.513953,-0.032572,-0.095647,-0.838349,-0.253009,-0.757088,-1.033035,-0.199131,-0.250806,-0.931289,0.142327,-1.011971,-0.895400,-0.742741,-0.699388,2.257867,-0.099340,1.667787,-0.054813,-0.431488,0.884635,1.129622,2.410938,-1.643719,0.133479,-0.142830,0.259798,-0.748254,-0.987031,-0.798986,0.267572,0.257405,-0.054241
5,0.376546,0.074094,-0.210281,1.435328,0.339802,1.033536,0.842556,-0.564369,0.467690,-0.305351,-0.540580,1.229228,-0.095647,1.089629,-0.253009,1.246785,0.182467,-0.199131,-0.250806,-0.931289,0.284151,0.343282,0.296734,-0.742741,-0.014372,-0.362343,-0.099340,-0.285269,-0.054813,1.817251,0.884635,1.129622,-0.248625,0.623540,0.133479,0.903237,0.259798,0.816222,-0.987031,1.455842,0.267572,0.257405,-0.054241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,-1.104339,0.000000,-0.154259,-0.031580,0.339802,-1.827163,-1.646773,-0.564369,-1.036517,-0.305351,-0.199298,-0.794860,-0.095647,-0.838349,-0.253009,-0.757088,-1.033035,-0.199131,-1.557024,0.611682,0.086527,-1.011971,-1.324954,-0.742741,0.792424,-0.362343,-0.099340,0.418535,-0.054813,-0.431488,0.884635,-0.662480,-0.248625,-0.510089,0.133479,-2.234964,-3.849149,-2.312730,1.254384,-0.798986,-1.178551,-2.653078,-0.054241
760,0.517780,0.000000,0.441256,-2.231943,-0.551357,-0.514607,-1.402721,-0.564369,-1.036517,-0.305351,-2.487327,-0.794860,-0.095647,-0.838349,-0.253009,-0.757088,1.397968,-0.199131,1.055412,0.611682,0.177202,0.343282,-0.070076,-0.742741,-0.699388,-0.362343,-0.099340,-0.285269,-0.054813,-0.056698,0.884635,-0.662480,-0.248625,-3.910978,-5.187781,0.903237,0.259798,-0.748254,1.254384,-0.798986,0.267572,0.257405,-0.054241
761,1.076487,0.655313,-0.473117,-0.765034,0.339802,-0.716539,-1.646773,-0.564369,-0.372140,-0.305351,-0.081532,1.574862,-0.095647,-0.838349,-0.253009,-0.757088,1.397968,4.737030,1.055412,-0.931289,0.163252,1.698534,1.952207,-0.742741,-0.699388,-0.362343,-0.099340,-0.285269,-0.054813,0.692882,-0.639314,-0.662480,-0.248625,-0.510089,0.133479,-1.188897,-3.849149,-0.748254,-0.987031,-0.798986,0.267572,0.257405,-0.054241
762,0.74417

In [38]:
scaled_df = pd.concat([scaled_df_num,df_cat_dummy],axis=1)
scaled_df2 = pd.concat([scaled_df_num2,df_cat_dummy2],axis=1)

scaled_df2

,GrLivArea,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,TotalBsmtSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,CentralAir,KitchenQual,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_None,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_None,BsmtFinType1_Rec,BsmtFinType1_Unf,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None
1,-1.303728,0.000000,-0.273213,-0.031580,0.339802,-1.053092,-1.646773,-0.564369,-0.479750,-0.305351,-0.430024,-0.794860,-0.095647,1.089629,-0.253009,-0.757088,-1.033035,-0.199131,-1.557024,0.611682,0.140002,0.343282,-0.325878,-0.742741,-0.699388,-0.362343,-0.099340,2.635518,-0.054813,-1.181068,1.646610,-0.662480,-0.248625,-0.510089,0.133479,-1.188897,0.259798,-0.748254,1.254384,-0.798986,0.267572,0.257405,-0.054241,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0
2,-0.902872,-1.282083,-0.724230,-0.765034,-0.551357,0.461396,0.012780,0.292857,0.254809,1.947821,0.033831,-0.794860,-0.095647,1.089629,-0.253009,-0.757088,-1.033035,-0.199131,-0.903915,-0.931289,0.244626,-1.011971,-0.967797,-0.742741,0.898982,-0.362343,-0.099340,-0.285269,-0.054813,-1.555858,0.884635,1.129622,-0.248625,0.623540,0.133479,-1.188897,0.259798,0.816222,-0.987031,1.455842,0.267572,0.257405,-0.054241,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0
3,-1.002567,-0.410255,-0.499030,-0.765034,3.013279,-1.355989,1.135419,-0.564369,0.687590,-0.305351,-0.475688,-0.794860,-0.095647,-0.838349,-0.253009,-0.757088,-1.033035,-0.199131,-0.903915,-0.931289,0.119077,-1.011971,-1.209119,0.451659,-0.699388,0.292709,3.300038,-0.285269,-0.054813,1.817251,-0.639314,1.129622,-0.248625,-0.510089,0.133479,0.903237,0.259798,0.816222,-0.987031,-0.798986,0.267572,-2.653078,-0.054241,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,-0.923642,0.558444,-0.241624,-1.498489,2.122120,-2.365648,0.940177,-0.564369,-1.036517,-0.305351,-1.513953,-0.032572,-0.095647,-0.838349,-0.253009,-0.757088,-1.033035,-0.199131,-0.250806,-0.931289,0.142327,-1.011971,-0.895400,-0.742741,-0.699388,2.257867,-0.099340,1.667787,-0.054813,-0.431488,0.884635,1.129622,2.410938,-1.643719,0.133479,-0.142830,0.259798,-0.748254,-0.987031,-0.798986,0.267572,0.257405,-0.054241,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
5,0.376546,0.074094,-0.210281,1.435328,0.339802,1.033536,0.842556,-0.564369,0.467690,-0.305351,-0.540580,1.229228,-0.095647,1.089629,-0.253009,1.246785,0.182467,-0.199131,-0.250806,-0.931289,0.284151,0.343282,0.296734,-0.742741,-0.014372,-0.362343,-0.099340,-0.285269,-0.054813,1.817251,0.884635,1.129622,-0.248625,0.623540,0.133479,0.903237,0.259798,0.816222,-0.987031,1.455842,0.267572,0.257405,-0.054241,0,0,0,1

In [39]:
x = scaled_df
y = price
x2 = scaled_df2

In [40]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mlr = LinearRegression()

In [41]:
cross_val_score_list1 = cross_val_score(mlr, x, y, cv=kf)
model_mean_score1 = np.mean(cross_val_score_list1)
model_std1 = np.std(cross_val_score_list1)
print('Avg Score: ',model_mean_score1)
print('Standard Deviation: ',model_std1)
print(cross_val_score_list1)


Avg Score:  0.907961478184936
Standard Deviation:  0.006945416755875572
[0.90704605 0.8969173  0.91881023 0.90833    0.90870382]


In [42]:
cross_val_score_list2 = cross_val_score(mlr, x2, y, cv=kf)
model_mean_score2 = np.mean(cross_val_score_list2)
model_std2 = np.std(cross_val_score_list2)
print('Avg Score: ',model_mean_score2)
print('Standard Deviation: ',model_std2)
print(cross_val_score_list2)


Avg Score:  0.908811160880797
Standard Deviation:  0.006981682807577999
[0.90740474 0.90030251 0.92025022 0.91236952 0.90372881]


In [43]:
# Initialize lists to store results
scores = []
coefficients_list = []
fold_num = 1


# Perform Grid Search in each fold
for train_index, test_index in kf.split(x):
    
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    
    mlr.fit(X_train, y_train)
    fold_score = mlr.score(X_test, y_test)
    
    scores.append({
        'score':fold_score,
        'fold':fold_num})
    # Store coefficients of the best model
    coefficients_list.append({
        'coef': mlr.coef_,
        'fold':fold_num})
    fold_num +=1

In [44]:
scores

[{'score': 0.9070460518732288, 'fold': 1},
 {'score': 0.8969173006735993, 'fold': 2},
 {'score': 0.9188102263108412, 'fold': 3},
 {'score': 0.9083299953052932, 'fold': 4},
 {'score': 0.908703816761717, 'fold': 5}]

In [45]:
coefficients_list

[{'coef': array([ 2.18904549e+04, -5.48886794e+03,  4.70417772e+01,  5.09310522e+03,
          1.14567527e+04,  6.76145777e+03,  8.32647964e+03,  8.35648195e+02,
          4.30093407e+03,  8.00207310e+03,  1.21163515e+03,  9.14480615e+03,
          2.92345624e+03,  7.39479376e+01,  1.28407999e+03, -4.72521910e+01,
          1.29749205e+03, -4.43815690e+03, -1.27905034e+03,  2.77857880e+03,
          2.50241119e+03, -2.30179831e+03,  1.74635420e+03,  3.67381961e+03,
          3.38049074e+02, -5.45867285e+02,  2.21804356e+03, -2.83569316e+03,
         -6.10893872e+02, -6.22395595e+02,  5.18368011e+03,  4.71452851e+03,
          3.10525847e+02,  1.91391201e+03, -8.15194770e+02,  3.93329586e+03,
         -8.35509408e+01,  8.00027974e+02,  4.32194826e+03,  1.46588188e+04,
          1.26800419e+04,  7.46038536e+02, -5.50988486e+03, -1.01878581e+04,
          1.35230315e+04, -7.46328719e+03, -8.49302971e+03,  3.90672938e+03,
          9.78498341e+04, -6.08539828e+03,  2.37323819e+04,  1.01779

In [46]:
total_score = 0
for i, el in enumerate(scores):
    total_score += el['score']
    avg_score = total_score / (i+1)
print(avg_score)

0.907961478184936


In [47]:
# Initialize lists to store results
scores2 = []
coefficients_list2 = []
fold_num = 1


# Perform Grid Search in each fold
for train_index, test_index in kf.split(x2):
    
    X_train, X_test = x2.iloc[train_index], x2.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    
    mlr.fit(X_train, y_train)
    fold_score = mlr.score(X_test, y_test)
    
    scores2.append({
        'score':fold_score,
        'fold':fold_num})
    # Store coefficients of the best model
    coefficients_list2.append({
        'coef': mlr.coef_,
        'fold':fold_num})
    fold_num +=1

In [48]:
scores2

[{'score': 0.9074047390324846, 'fold': 1},
 {'score': 0.9003025089238194, 'fold': 2},
 {'score': 0.9202502200533899, 'fold': 3},
 {'score': 0.9123695242666038, 'fold': 4},
 {'score': 0.9037288121276879, 'fold': 5}]

In [49]:
total_score = 0
for i, el in enumerate(scores2):
    total_score += el['score']
    avg_score = total_score / (i+1)
print(avg_score)

0.908811160880797
